该notebook中的模型，可以做到与biogeme的结果基本一致，包括对于模型的Structural equation，Measurement equation以及Utility function中的参数的估计，对应研究进展20181023。

# Data and Package Prepare

In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from tensorflow.keras import backend as K
from tensorflow.keras.constraints import min_max_norm

In [2]:
train_data = pd.read_csv('optif.csv', low_memory=False)
#train_data = pd.read_csv('D:/ProFile/Latent/biogeme/optima_keras.csv', low_memory=False)
random_num = 500
# S=tf.Session()

In [3]:
trainable = True

In [4]:
train_latent = train_data[['ContIncome_0_4000','ContIncome_10000_more', 'ContIncome_4000_6000', 
                           'ContIncome_6000_8000', 'ContIncome_8000_10000', 
                           'age_65_more','haveChildren', 
                           'haveGA', 'highEducation', 'individualHouse', 'male',
                           'moreThanOneBike', 'moreThanOneCar']]

train_PTcom = train_data[['CostPT_HWH', 'CostPT_Other']]
train_CARcom = train_data[['CostCar_HWH', 'CostCar_Other']]

train_PTasc = train_data[['TimePT', 'WaitingTimePT']]
train_CARasc = train_data[['TimeCar']]
train_SMasc = train_data[['distance_km']]

train_indicator = train_data[['Envir01','Envir02','Envir03','Mobil11','Mobil14','Mobil16','Mobil17']]

train_y = train_data[['Choice']]

# Latent Variable Submodel

In [5]:
class LL(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(LL, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = (train_indicator.shape[1], )
        # Create a trainable weight variable for this layer.
        self.sigma = self.add_weight(name='sigma',
                                      shape=shape,
                                     initializer=keras.initializers.Constant(value=np.ones(train_indicator.shape[1])),
#                                      initializer=keras.initializers.Constant(
#                                         value=np.array([1.25,1.12,1.07,1.08,1.05,1.1,1.11])),
                                      trainable=trainable)
        # Be sure to call this at the end
        super(LL, self).build(input_shape)

    def call(self, inputs):
#         error = tf.subtract(inputs[0],inputs[1])
#         se=K.square(error)
        se=K.square(inputs)
        term1 = tf.divide(se, tf.multiply(2., K.square(self.sigma)))
        term2 = K.log(self.sigma)
        term3 = tf.multiply(0.5, K.log(6.283185))
        return tf.add(term1, tf.add(term2, term3))

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

    def get_config(self):
        base_config = super(LL, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [6]:
class Subtract(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(Subtract, self).__init__(**kwargs)

    def build(self, input_shape):

        # Be sure to call this at the end
        super(Subtract, self).build(input_shape)

    def call(self, inputs):
        return tf.cast(tf.subtract(inputs[0],inputs[1]),tf.float32)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

    def get_config(self):
        base_config = super(Subtract, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [7]:
def LL_loss(y_true, y_pred):
    return K.sum(K.sum(K.abs(y_pred - y_true), axis=0))

In [8]:
def multi_indicator_loss(y_true, y_pred):
    n=train_data.shape[0]
    k1,k2 = 14,16
    error2 = K.sum(K.abs(y_pred - y_true), axis=0)
    term1 = tf.multiply(0.5, n-k2)
    term2_1 = tf.multiply(0.5, tf.log(tf.divide(error2, n-k2)))
    term2_2 = tf.multiply(0.5, tf.log(6.283185))
    term2 = tf.multiply(tf.cast(n, tf.float32), tf.add(term2_1, term2_2))
    L2 = tf.add(term1, term2)

    return K.sum(L2)

In [9]:
# latent variable sub model
input_latent = keras.Input(shape = (train_latent.shape[1],), name = 'input_latent')
y_ture = keras.Input(shape = (train_indicator.shape[1],), name = 'input_y_ture')
# car_sys = keras.layers.Dense(1, name = 'car_sys', activation = 'linear',
#                             kernel_initializer=keras.initializers.Constant(
#                                 value=np.array([[ -0.0265 ],
#                                                 [-0.0863],
#                                                 [0.2],
#                                                 [-0.205],
#                                                 [0.541],
#                                                 [-0.077],
#                                                 [0.0182],
#                                                 [ 0.724],
#                                                 [ 0.288],
#                                                 [ 0.0764],
#                                                 [-0.0694],
#                                                 [ 0.293],
#                                                 [-0.68]])),
#                                bias_initializer=keras.initializers.Constant(value=2.27))(input_latent)
car_sys = keras.layers.Dense(1, name = 'car_sys', activation = 'linear')(input_latent)

indicator_ref = keras.layers.Dense(1, name = 'indicator_ref',activation = 'linear',
                               kernel_initializer=keras.initializers.Constant(value=1),
                               bias_initializer=keras.initializers.Constant(value=0))(car_sys)
indicator_other = keras.layers.Dense(train_indicator.shape[1]-1, 
                                     name = 'indicator_other',
                                   kernel_initializer=keras.initializers.RandomUniform(minval=-1., maxval=0., seed=None),
#                                      kernel_initializer=keras.initializers.Constant(
#                                          value=np.array([0.493,-0.706,-0.626,-0.707,-0.542,-0.456])),
#                                      bias_initializer=keras.initializers.Constant(
#                                          value=np.array([2.02,4.64,5.3,4.9,4.79,4.53,])),
                                     activation = 'linear')(car_sys)

indicator = keras.layers.Concatenate(axis=-1)([indicator_ref,indicator_other])

error = Subtract(1)([indicator, y_ture])

loglikelihood = LL(1)(error)

# idmodel=tf.keras.Model(inputs = [input_latent, y_ture],
#                     outputs = indicator)

idmodel=tf.keras.Model(inputs = [input_latent, y_ture],
                    outputs = loglikelihood)

#idmodel.layers[3].trainable = False
idmodel.layers[2].trainable = False
#idmodel.layers[7].trainable = False

idmodel.compile(optimizer=keras.optimizers.Adam(0.01),
              loss = LL_loss)

LLoutput = np.zeros((train_latent.shape[0],train_indicator.shape[1]))

In [12]:
idmodel.evaluate([train_latent, train_indicator], 
          LLoutput, batch_size=train_latent.shape[0])

1545/1545 [==============================] - 0s 266us/step


17448.8515625

In [13]:
idmodel.layers[1].get_weights()

[array([[ 0.5037607 ],
        [-0.5718652 ],
        [ 0.162583  ],
        [ 0.09169263],
        [ 0.23784637],
        [-0.03251243],
        [-0.3057918 ],
        [-0.1295259 ],
        [-0.20398897],
        [ 0.326123  ],
        [ 0.59025836],
        [ 0.3269226 ],
        [-0.156034  ]], dtype=float32), array([0.], dtype=float32)]

In [14]:
idmodel.layers[3].get_weights()

[array([[-0.0546428 , -0.24449205, -0.41301572, -0.53613544, -0.8410039 ,
         -0.8028412 ]], dtype=float32),
 array([0., 0., 0., 0., 0., 0.], dtype=float32)]

In [15]:
idmodel.layers[7].get_weights()

[array([1., 1., 1., 1., 1., 1., 1.], dtype=float32)]

In [11]:
idmodel.fit([train_latent, train_indicator], 
         LLoutput, epochs=8000, batch_size = train_latent.shape[0])

Epoch 1/8000
1545/1545 [==============================] - 1s 707us/step - loss: 69817.8281
Epoch 2/8000
1545/1545 [==============================] - 0s 30us/step - loss: 68098.2344
Epoch 3/8000
1545/1545 [==============================] - 0s 20us/step - loss: 66208.8125
Epoch 4/8000
1545/1545 [==============================] - 0s 26us/step - loss: 64318.3633
Epoch 5/8000
1545/1545 [==============================] - 0s 31us/step - loss: 62488.8789
Epoch 6/8000
1545/1545 [==============================] - 0s 19us/step - loss: 60749.5195
Epoch 7/8000
1545/1545 [==============================] - 0s 21us/step - loss: 59114.7656
Epoch 8/8000
1545/1545 [==============================] - 0s 21us/step - loss: 57590.7227
Epoch 9/8000
1545/1545 [==============================] - 0s 30us/step - loss: 56177.6680
Epoch 10/8000
1545/1545 [==============================] - 0s 19us/step - loss: 54871.4375
Epoch 11/8000
1545/1545 [==============================] - 0s 29us/step - loss: 53664.2422
Epoch 1

Epoch 92/8000
1545/1545 [==============================] - 0s 25us/step - loss: 26026.2012
Epoch 93/8000
1545/1545 [==============================] - 0s 20us/step - loss: 25944.1973
Epoch 94/8000
1545/1545 [==============================] - 0s 30us/step - loss: 25863.8340
Epoch 95/8000
1545/1545 [==============================] - 0s 21us/step - loss: 25785.0566
Epoch 96/8000
1545/1545 [==============================] - 0s 22us/step - loss: 25707.8203
Epoch 97/8000
1545/1545 [==============================] - 0s 21us/step - loss: 25632.0781
Epoch 98/8000
1545/1545 [==============================] - 0s 23us/step - loss: 25557.7930
Epoch 99/8000
1545/1545 [==============================] - 0s 19us/step - loss: 25484.9258
Epoch 100/8000
1545/1545 [==============================] - 0s 25us/step - loss: 25413.4395
Epoch 101/8000
1545/1545 [==============================] - 0s 27us/step - loss: 25343.3066
Epoch 102/8000
1545/1545 [==============================] - 0s 22us/step - loss: 25274.4

Epoch 182/8000
1545/1545 [==============================] - 0s 32us/step - loss: 22180.2207
Epoch 183/8000
1545/1545 [==============================] - 0s 21us/step - loss: 22158.5801
Epoch 184/8000
1545/1545 [==============================] - 0s 22us/step - loss: 22137.1758
Epoch 185/8000
1545/1545 [==============================] - 0s 97us/step - loss: 22116.0039
Epoch 186/8000
1545/1545 [==============================] - 0s 26us/step - loss: 22095.0547
Epoch 187/8000
1545/1545 [==============================] - 0s 28us/step - loss: 22074.3320
Epoch 188/8000
1545/1545 [==============================] - 0s 35us/step - loss: 22053.8281
Epoch 189/8000
1545/1545 [==============================] - 0s 26us/step - loss: 22033.5391
Epoch 190/8000
1545/1545 [==============================] - 0s 22us/step - loss: 22013.4648
Epoch 191/8000
1545/1545 [==============================] - 0s 20us/step - loss: 21993.6016
Epoch 192/8000
1545/1545 [==============================] - 0s 35us/step - loss:

1545/1545 [==============================] - 0s 28us/step - loss: 20882.6719
Epoch 272/8000
1545/1545 [==============================] - 0s 23us/step - loss: 20873.0117
Epoch 273/8000
1545/1545 [==============================] - 0s 25us/step - loss: 20863.4258
Epoch 274/8000
1545/1545 [==============================] - 0s 27us/step - loss: 20853.9102
Epoch 275/8000
1545/1545 [==============================] - 0s 22us/step - loss: 20844.4688
Epoch 276/8000
1545/1545 [==============================] - 0s 19us/step - loss: 20835.0977
Epoch 277/8000
1545/1545 [==============================] - 0s 23us/step - loss: 20825.7969
Epoch 278/8000
1545/1545 [==============================] - 0s 23us/step - loss: 20816.5664
Epoch 279/8000
1545/1545 [==============================] - 0s 24us/step - loss: 20807.4023
Epoch 280/8000
1545/1545 [==============================] - 0s 19us/step - loss: 20798.3047
Epoch 281/8000
1545/1545 [==============================] - 0s 30us/step - loss: 20789.2773
Epo

Epoch 361/8000
1545/1545 [==============================] - 0s 23us/step - loss: 20232.4961
Epoch 362/8000
1545/1545 [==============================] - 0s 21us/step - loss: 20227.1211
Epoch 363/8000
1545/1545 [==============================] - 0s 20us/step - loss: 20221.7754
Epoch 364/8000
1545/1545 [==============================] - 0s 48us/step - loss: 20216.4590
Epoch 365/8000
1545/1545 [==============================] - 0s 21us/step - loss: 20211.1758
Epoch 366/8000
1545/1545 [==============================] - 0s 27us/step - loss: 20205.9199
Epoch 367/8000
1545/1545 [==============================] - 0s 20us/step - loss: 20200.6914
Epoch 368/8000
1545/1545 [==============================] - 0s 21us/step - loss: 20195.4941
Epoch 369/8000
1545/1545 [==============================] - 0s 24us/step - loss: 20190.3262
Epoch 370/8000
1545/1545 [==============================] - 0s 22us/step - loss: 20185.1875
Epoch 371/8000
1545/1545 [==============================] - 0s 43us/step - loss:

...

Epoch 7530/8000
1545/1545 [==============================] - 0s 20us/step - loss: 17448.8535
Epoch 7531/8000
1545/1545 [==============================] - 0s 21us/step - loss: 17448.8535
Epoch 7532/8000
1545/1545 [==============================] - 0s 20us/step - loss: 17448.8535
Epoch 7533/8000
1545/1545 [==============================] - 0s 28us/step - loss: 17448.8535
Epoch 7534/8000
1545/1545 [==============================] - 0s 20us/step - loss: 17448.8535
Epoch 7535/8000
1545/1545 [==============================] - 0s 49us/step - loss: 17448.8535
Epoch 7536/8000
1545/1545 [==============================] - 0s 23us/step - loss: 17448.8535
Epoch 7537/8000
1545/1545 [==============================] - 0s 48us/step - loss: 17448.8535
Epoch 7538/8000
1545/1545 [==============================] - 0s 22us/step - loss: 17448.8535
Epoch 7539/8000
1545/1545 [==============================] - 0s 21us/step - loss: 17448.8535
Epoch 7540/8000
1545/1545 [==============================] - 0s 24us/s

1545/1545 [==============================] - 0s 21us/step - loss: 17448.9219
Epoch 7619/8000
1545/1545 [==============================] - 0s 22us/step - loss: 17448.8750
Epoch 7620/8000
1545/1545 [==============================] - 0s 21us/step - loss: 17448.8535
Epoch 7621/8000
1545/1545 [==============================] - 0s 23us/step - loss: 17448.8691
Epoch 7622/8000
1545/1545 [==============================] - 0s 22us/step - loss: 17448.8945
Epoch 7623/8000
1545/1545 [==============================] - 0s 22us/step - loss: 17448.8984
Epoch 7624/8000
1545/1545 [==============================] - 0s 27us/step - loss: 17448.8770
Epoch 7625/8000
1545/1545 [==============================] - 0s 26us/step - loss: 17448.8555
Epoch 7626/8000
1545/1545 [==============================] - 0s 27us/step - loss: 17448.8555
Epoch 7627/8000
1545/1545 [==============================] - 0s 21us/step - loss: 17448.8730
Epoch 7628/8000
1545/1545 [==============================] - 0s 28us/step - loss: 1744

Epoch 7707/8000
1545/1545 [==============================] - 0s 23us/step - loss: 17448.8516
Epoch 7708/8000
1545/1545 [==============================] - 0s 25us/step - loss: 17448.8516
Epoch 7709/8000
1545/1545 [==============================] - 0s 22us/step - loss: 17448.8516
Epoch 7710/8000
1545/1545 [==============================] - 0s 64us/step - loss: 17448.8535
Epoch 7711/8000
1545/1545 [==============================] - 0s 22us/step - loss: 17448.8516
Epoch 7712/8000
1545/1545 [==============================] - 0s 24us/step - loss: 17448.8516
Epoch 7713/8000
1545/1545 [==============================] - 0s 20us/step - loss: 17448.8535
Epoch 7714/8000
1545/1545 [==============================] - 0s 22us/step - loss: 17448.8516
Epoch 7715/8000
1545/1545 [==============================] - 0s 19us/step - loss: 17448.8535
Epoch 7716/8000
1545/1545 [==============================] - 0s 26us/step - loss: 17448.8535
Epoch 7717/8000
1545/1545 [==============================] - 0s 28us/s

1545/1545 [==============================] - 0s 24us/step - loss: 17448.8535
Epoch 7796/8000
1545/1545 [==============================] - 0s 21us/step - loss: 17448.8535
Epoch 7797/8000
1545/1545 [==============================] - 0s 18us/step - loss: 17448.8535
Epoch 7798/8000
1545/1545 [==============================] - 0s 19us/step - loss: 17448.8535
Epoch 7799/8000
1545/1545 [==============================] - 0s 24us/step - loss: 17448.8535
Epoch 7800/8000
1545/1545 [==============================] - 0s 24us/step - loss: 17448.8535
Epoch 7801/8000
1545/1545 [==============================] - 0s 22us/step - loss: 17448.8535
Epoch 7802/8000
1545/1545 [==============================] - 0s 19us/step - loss: 17448.8535
Epoch 7803/8000
1545/1545 [==============================] - 0s 22us/step - loss: 17448.8535
Epoch 7804/8000
1545/1545 [==============================] - 0s 27us/step - loss: 17448.8535
Epoch 7805/8000
1545/1545 [==============================] - 0s 20us/step - loss: 1744

Epoch 7884/8000
1545/1545 [==============================] - 0s 22us/step - loss: 17449.1914
Epoch 7885/8000
1545/1545 [==============================] - 0s 24us/step - loss: 17448.8691
Epoch 7886/8000
1545/1545 [==============================] - 0s 22us/step - loss: 17449.3750
Epoch 7887/8000
1545/1545 [==============================] - 0s 28us/step - loss: 17449.8652
Epoch 7888/8000
1545/1545 [==============================] - 0s 22us/step - loss: 17449.6543
Epoch 7889/8000
1545/1545 [==============================] - 0s 21us/step - loss: 17449.0391
Epoch 7890/8000
1545/1545 [==============================] - 0s 25us/step - loss: 17448.8828
Epoch 7891/8000
1545/1545 [==============================] - 0s 27us/step - loss: 17449.2578
Epoch 7892/8000
1545/1545 [==============================] - 0s 21us/step - loss: 17449.4590
Epoch 7893/8000
1545/1545 [==============================] - 0s 33us/step - loss: 17449.1680
Epoch 7894/8000
1545/1545 [==============================] - 0s 21us/s

In [16]:
i = 0
for layer in idmodel.layers:
    print(layer.name, i)
    i += 1

input_latent 0
car_sys 1
indicator_ref 2
indicator_other 3
concatenate 4
input_y_ture 5
subtract 6
ll 7


In [17]:
idmodel.layers[1].get_weights()

[array([[-0.1259092 ],
        [-0.09641296],
        [ 0.3678927 ],
        [-0.34928215],
        [ 0.625567  ],
        [-0.06615467],
        [ 0.05817849],
        [ 0.6222404 ],
        [ 0.21793641],
        [ 0.0993166 ],
        [-0.04281182],
        [ 0.279273  ],
        [-0.61859584]], dtype=float32), array([2.5601623], dtype=float32)]

In [18]:
idmodel.layers[2].get_weights()

[array([[1.]], dtype=float32), array([0.], dtype=float32)]

In [19]:
idmodel.layers[3].get_weights()

[array([[ 0.46806106, -0.77926636, -0.7596789 , -0.70193034, -0.65280664,
         -0.7071629 ]], dtype=float32),
 array([2.0837853, 5.022559 , 5.7847285, 5.0839744, 5.1904716, 5.5095224],
       dtype=float32)]

In [20]:
idmodel.layers[7].get_weights()

[array([1.3653334, 1.1685717, 1.2034935, 1.1288774, 1.1913147, 1.14992  ,
        1.3085434], dtype=float32)]

# Discrete Choice Submodel

计算效用中公共系数部分

V0 alternative specific part

In [13]:
# input [carlover, time_PT_scaled, waitingtimePT]
class V0_as(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(V0_as, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = tf.TensorShape((1, self.output_dim))
        # Create a trainable weight variable for this layer.
        self.beta_time_PT = self.add_weight(name='beta_time_PT',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=-1.72),
                                      # constraints = min_max_norm(min_value=-1.020001, max_value=-1.019999, rate=1.0, axis=0),
                                      trainable=trainable)
        self.beta_cl_PT = self.add_weight(name='beta_cl_PT',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=1.37),
                                      # constraints = min_max_norm(min_value=1.929999, max_value=1.930001, rate=1.0, axis=0),
                                      trainable=trainable)
        self.beta_waiting_time = self.add_weight(name='self.beta_waiting_time',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=-0.0275),
                                      # constraints = min_max_norm(min_value=-0.075001, max_value=-0.074999, rate=1.0, axis=0),           
                                      trainable=trainable)
        # Be sure to call this at the end
        super(V0_as, self).build(input_shape)

    def call(self, inputs):
        cl = tf.multiply(inputs[0], self.beta_cl_PT)
        timePT = tf.transpose(tf.multiply(self.beta_time_PT, inputs[1][:,0]))
        waitingtime = tf.transpose(tf.multiply(inputs[1][:,1], self.beta_waiting_time))
        # return tf.transpose(tf.add(timePT, waitingtime))
        return tf.add(timePT, tf.add(waitingtime, cl))
        # return [timePT, waitingtime]

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


    def get_config(self):
        base_config = super(V0_as, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)

V1 alternative specific part

In [14]:
# input [carlover, time_CAR_scaled]
class V1_as(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(V1_as, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = tf.TensorShape((1, self.output_dim))
        # Create a trainable weight variable for this layer.
        self.asc_CAR = self.add_weight(name='asc_CAR',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=4.66),
                                      # constraints = min_max_norm(min_value=0.740999, max_value=0.741001, rate=1.0, axis=0),
                                      trainable=trainable)
        
        self.beta_time_CAR = self.add_weight(name='beta_time_CAR',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=-5.83),
                                      # constraints = min_max_norm(min_value=0.482999, max_value=0.483001, rate=1.0, axis=0),          
                                      trainable=trainable)
        # Be sure to call this at the end
        super(V1_as, self).build(input_shape)

    def call(self, inputs):
        # cl = tf.multiply(inputs[0], self.beta_cl_CAR)
        
        timeCAR = tf.multiply(inputs[1], self.beta_time_CAR)
        
        # return tf.transpose(tf.add(timeCAR, self.asc_CAR))
        # return tf.add(tf.add(cl,timeCAR), self.asc_CAR)
        return tf.add(timeCAR, self.asc_CAR)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


    def get_config(self):
        base_config = super(V1_as, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)

V2 alternative specific part

In [15]:
class V2_as(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(V2_as, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = tf.TensorShape((1, self.output_dim))
        # Create a trainable weight variable for this layer.
        self.beta_dist = self.add_weight(name='beta_dist',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=-1.22),
                                      # constraints = min_max_norm(min_value=-4.88001, max_value=-4.87999, rate=1.0, axis=0),   
                                      trainable=trainable)
        self.asc_SM = self.add_weight(name='asc_SM',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=-1.62),
                                      # constraints = min_max_norm(min_value=1.23999, max_value=1.24001, rate=1.0, axis=0), 
                                      trainable=trainable)
        self.beta_cl_SM = self.add_weight(name='beta_cl_SM',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=2.),
                                      # constraints = min_max_norm(min_value=-20.4001, max_value=-20.3999, rate=1.0, axis=0),
                                      trainable=trainable)
        # Be sure to call this at the end
        super(V2_as, self).build(input_shape)

    def call(self, inputs):
        cl_SM = tf.multiply(inputs[0], self.beta_cl_SM)
        dist = tf.multiply(inputs[1], self.beta_dist)
        
        return tf.add(tf.add(cl_SM, dist), self.asc_SM)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


    def get_config(self):
        base_config = super(V2_as, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [16]:
class TransToRandShape(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(TransToRandShape, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = tf.TensorShape((1, self.output_dim))
        # Be sure to call this at the end
        super(TransToRandShape, self).build(input_shape)

    def call(self, inputs):
               
        return tf.add(inputs,tf.zeros((tf.shape(inputs)[0],random_num)))

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


    def get_config(self):
        base_config = super(TransToRandShape, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [17]:
class Add_Random(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(Add_Random, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = tf.TensorShape((1, self.output_dim))
        # Be sure to call this at the end
        super(Add_Random, self).build(input_shape)

    def call(self, inputs):
               
        return tf.add(inputs,tf.random_normal((tf.shape(inputs)[0],random_num)))

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


    def get_config(self):
        base_config = super(Add_Random, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)

Model Specification

In [18]:
### input layers
# batch_size = 32

input_latent = keras.Input(shape = (train_latent.shape[1],), name = 'input_latent')
input_PTcom = keras.Input(shape = (train_PTcom.shape[1],), name = 'input_PTcom')
input_CARcom = keras.Input(shape = (train_CARcom.shape[1],), name = 'input_CARcom')
input_PTasc = keras.Input(shape = (train_PTasc.shape[1],), name = 'input_PTasc')
input_CARasc = keras.Input(shape = (train_CARasc.shape[1],), name = 'input_CARasc')
input_SMasc = keras.Input(shape = (train_SMasc.shape[1],), name = 'input_SMasc')

### car lover
idmodel.layers[1].trainable = False
car_sys = idmodel.layers[1](input_latent)
# carlover = keras.layers.Lambda(lambda x: x + tf.random_normal((1505,random_num)), name = 'carlover')(car_sys)
carlover = Add_Random(1,name = 'carlover')(car_sys)

### common coefficients part in utility
utilcommon = keras.layers.Dense(1, name = 'util_common', use_bias=False)
PTcom = TransToRandShape(1)(utilcommon(input_PTcom))
CARcom = TransToRandShape(1)(utilcommon(input_CARcom))


### alternative specific part in utility
# V0
# util_input0 = keras.layers.concatenate([carlover, input_PTasc], axis = -1)
util_V0as = V0_as(1)([carlover, input_PTasc])
V0 = keras.layers.Add(name = 'V0')([util_V0as, PTcom])

# V1
# util_input1 = keras.layers.concatenate([carlover, input_CARasc], axis = -1)
util_V1as = V1_as(1)([carlover, input_CARasc])
V1 = keras.layers.Add(name = 'V1')([util_V1as, CARcom])

# V2
_V2 = V2_as(1)([carlover, input_SMasc])
V2 =keras.layers.Lambda(lambda x: x + tf.zeros((1,random_num)), name = 'V2')(_V2)
### merge
# merged_vector=keras.layers.concatenate([[V0], [V1], [V2]], axis=-1)

prediction =keras.layers.Lambda(lambda x: tf.transpose(tf.reduce_mean(tf.nn.softmax(x, axis = 0),axis=2)), 
                                name = 'prediction')([V0, V1, V2])

model=tf.keras.Model(inputs = [input_latent, input_PTcom, input_CARcom, input_PTasc, input_CARasc, input_SMasc],
                    outputs = prediction)


model.compile(optimizer=keras.optimizers.Adam(0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']
)


In [19]:
model.evaluate([train_latent, train_PTcom, train_CARcom, train_PTasc, train_CARasc, train_SMasc], 
          train_y, batch_size=train_latent.shape[0])

1545/1545 [==============================] - 2s 979us/step


[14.734243392944336, 0.05760517716407776]

In [20]:
# tf.keras.backend.get_value(util_V1as)
# callbacks = keras.callbacks.TensorBoard(log_dir='./logs')
model.fit([train_latent, train_PTcom, train_CARcom, train_PTasc, train_CARasc, train_SMasc], 
          train_y, epochs=4000, batch_size = train_latent.shape[0])

Epoch 1/4000
1545/1545 [==============================] - 2s 2ms/step - loss: 14.7340 - acc: 0.0583
Epoch 2/4000
1545/1545 [==============================] - 0s 181us/step - loss: 14.7304 - acc: 0.0583
Epoch 3/4000
1545/1545 [==============================] - 0s 192us/step - loss: 14.7283 - acc: 0.0583
Epoch 4/4000
1545/1545 [==============================] - 0s 176us/step - loss: 14.7227 - acc: 0.0576
Epoch 5/4000
1545/1545 [==============================] - 0s 137us/step - loss: 14.7184 - acc: 0.0576
Epoch 6/4000
1545/1545 [==============================] - 0s 188us/step - loss: 14.7121 - acc: 0.0576
Epoch 7/4000
1545/1545 [==============================] - 0s 213us/step - loss: 14.7076 - acc: 0.0583
Epoch 8/4000
1545/1545 [==============================] - 0s 174us/step - loss: 14.7000 - acc: 0.0589
Epoch 9/4000
1545/1545 [==============================] - 0s 176us/step - loss: 14.6946 - acc: 0.0589
Epoch 10/4000
1545/1545 [==============================] - 0s 180us/step - loss: 14.

Epoch 81/4000
1545/1545 [==============================] - 0s 165us/step - loss: 10.5887 - acc: 0.2576
Epoch 82/4000
1545/1545 [==============================] - 0s 152us/step - loss: 10.5297 - acc: 0.2634
Epoch 83/4000
1545/1545 [==============================] - 0s 161us/step - loss: 10.4835 - acc: 0.2731
Epoch 84/4000
1545/1545 [==============================] - 0s 173us/step - loss: 10.4406 - acc: 0.2770
Epoch 85/4000
1545/1545 [==============================] - 0s 147us/step - loss: 10.4038 - acc: 0.2816
Epoch 86/4000
1545/1545 [==============================] - 0s 142us/step - loss: 10.3775 - acc: 0.2880
Epoch 87/4000
1545/1545 [==============================] - 0s 166us/step - loss: 10.3575 - acc: 0.2887
Epoch 88/4000
1545/1545 [==============================] - 0s 277us/step - loss: 10.3440 - acc: 0.2919
Epoch 89/4000
1545/1545 [==============================] - 0s 191us/step - loss: 10.3306 - acc: 0.2900
Epoch 90/4000
1545/1545 [==============================] - 0s 244us/step 

1545/1545 [==============================] - 0s 220us/step - loss: 9.7411 - acc: 0.3081
Epoch 161/4000
1545/1545 [==============================] - 0s 197us/step - loss: 9.7193 - acc: 0.3081
Epoch 162/4000
1545/1545 [==============================] - 0s 312us/step - loss: 9.6967 - acc: 0.3094
Epoch 163/4000
1545/1545 [==============================] - 1s 349us/step - loss: 9.6765 - acc: 0.3120
Epoch 164/4000
1545/1545 [==============================] - 0s 275us/step - loss: 9.6514 - acc: 0.3133
Epoch 165/4000
1545/1545 [==============================] - 0s 280us/step - loss: 9.6277 - acc: 0.3139
Epoch 166/4000
1545/1545 [==============================] - 1s 337us/step - loss: 9.6028 - acc: 0.3139
Epoch 167/4000
1545/1545 [==============================] - 0s 283us/step - loss: 9.5750 - acc: 0.3139
Epoch 168/4000
1545/1545 [==============================] - 0s 294us/step - loss: 9.5486 - acc: 0.3152
Epoch 169/4000
1545/1545 [==============================] - 1s 348us/step - loss: 9.5192

Epoch 240/4000
1545/1545 [==============================] - 0s 151us/step - loss: 2.9537 - acc: 0.6964
Epoch 241/4000
1545/1545 [==============================] - 0s 263us/step - loss: 2.9332 - acc: 0.6971
Epoch 242/4000
1545/1545 [==============================] - 0s 214us/step - loss: 2.9171 - acc: 0.6984
Epoch 243/4000
1545/1545 [==============================] - 0s 215us/step - loss: 2.9035 - acc: 0.6990
Epoch 244/4000
1545/1545 [==============================] - 0s 172us/step - loss: 2.8853 - acc: 0.7010
Epoch 245/4000
1545/1545 [==============================] - 0s 171us/step - loss: 2.8704 - acc: 0.6997
Epoch 246/4000
1545/1545 [==============================] - 0s 220us/step - loss: 2.8530 - acc: 0.6990
Epoch 247/4000
1545/1545 [==============================] - 0s 239us/step - loss: 2.8379 - acc: 0.7010
Epoch 248/4000
1545/1545 [==============================] - 0s 185us/step - loss: 2.8235 - acc: 0.7016
Epoch 249/4000
1545/1545 [==============================] - 0s 245us/step

Epoch 320/4000
1545/1545 [==============================] - 0s 196us/step - loss: 2.0992 - acc: 0.7172
Epoch 321/4000
1545/1545 [==============================] - 0s 205us/step - loss: 2.0905 - acc: 0.7165
Epoch 322/4000
1545/1545 [==============================] - 0s 164us/step - loss: 2.0814 - acc: 0.7159
Epoch 323/4000
1545/1545 [==============================] - 0s 238us/step - loss: 2.0745 - acc: 0.7178
Epoch 324/4000
1545/1545 [==============================] - 0s 174us/step - loss: 2.0683 - acc: 0.7152
Epoch 325/4000
1545/1545 [==============================] - 0s 180us/step - loss: 2.0662 - acc: 0.7159
Epoch 326/4000
1545/1545 [==============================] - 0s 183us/step - loss: 2.0589 - acc: 0.7178
Epoch 327/4000
1545/1545 [==============================] - 0s 171us/step - loss: 2.0575 - acc: 0.7178
Epoch 328/4000
1545/1545 [==============================] - 0s 143us/step - loss: 2.0541 - acc: 0.7184
Epoch 329/4000
1545/1545 [==============================] - 0s 164us/step

...

Epoch 3647/4000
1545/1545 [==============================] - 0s 158us/step - loss: 0.6068 - acc: 0.7385
Epoch 3648/4000
1545/1545 [==============================] - 0s 145us/step - loss: 0.6066 - acc: 0.7359
Epoch 3649/4000
1545/1545 [==============================] - 0s 148us/step - loss: 0.6069 - acc: 0.7353
Epoch 3650/4000
1545/1545 [==============================] - 0s 220us/step - loss: 0.6075 - acc: 0.7353
Epoch 3651/4000
1545/1545 [==============================] - 0s 170us/step - loss: 0.6074 - acc: 0.7366
Epoch 3652/4000
1545/1545 [==============================] - 0s 148us/step - loss: 0.6060 - acc: 0.7327
Epoch 3653/4000
1545/1545 [==============================] - 0s 158us/step - loss: 0.6067 - acc: 0.7385
Epoch 3654/4000
1545/1545 [==============================] - 0s 167us/step - loss: 0.6072 - acc: 0.7379
Epoch 3655/4000
1545/1545 [==============================] - 0s 158us/step - loss: 0.6069 - acc: 0.7346
Epoch 3656/4000
1545/1545 [==============================] - 0s 

Epoch 3726/4000
1545/1545 [==============================] - 0s 199us/step - loss: 0.6072 - acc: 0.7353
Epoch 3727/4000
1545/1545 [==============================] - 0s 186us/step - loss: 0.6086 - acc: 0.7359
Epoch 3728/4000
1545/1545 [==============================] - 0s 177us/step - loss: 0.6090 - acc: 0.7385
Epoch 3729/4000
1545/1545 [==============================] - 0s 185us/step - loss: 0.6070 - acc: 0.7379
Epoch 3730/4000
1545/1545 [==============================] - 0s 168us/step - loss: 0.6079 - acc: 0.7366
Epoch 3731/4000
1545/1545 [==============================] - 0s 154us/step - loss: 0.6093 - acc: 0.7398
Epoch 3732/4000
1545/1545 [==============================] - 0s 174us/step - loss: 0.6080 - acc: 0.7333
Epoch 3733/4000
1545/1545 [==============================] - 0s 173us/step - loss: 0.6084 - acc: 0.7372
Epoch 3734/4000
1545/1545 [==============================] - 0s 164us/step - loss: 0.6079 - acc: 0.7346
Epoch 3735/4000
1545/1545 [==============================] - 0s 

Epoch 3805/4000
1545/1545 [==============================] - 0s 199us/step - loss: 0.6071 - acc: 0.7366
Epoch 3806/4000
1545/1545 [==============================] - 0s 158us/step - loss: 0.6079 - acc: 0.7333
Epoch 3807/4000
1545/1545 [==============================] - 0s 185us/step - loss: 0.6075 - acc: 0.7366
Epoch 3808/4000
1545/1545 [==============================] - 0s 200us/step - loss: 0.6073 - acc: 0.7333
Epoch 3809/4000
1545/1545 [==============================] - 0s 164us/step - loss: 0.6061 - acc: 0.7392
Epoch 3810/4000
1545/1545 [==============================] - 0s 183us/step - loss: 0.6072 - acc: 0.7333
Epoch 3811/4000
1545/1545 [==============================] - 0s 169us/step - loss: 0.6075 - acc: 0.7372
Epoch 3812/4000
1545/1545 [==============================] - 0s 151us/step - loss: 0.6082 - acc: 0.7353
Epoch 3813/4000
1545/1545 [==============================] - 0s 151us/step - loss: 0.6070 - acc: 0.7379
Epoch 3814/4000
1545/1545 [==============================] - 0s 

Epoch 3884/4000
1545/1545 [==============================] - 0s 146us/step - loss: 0.6060 - acc: 0.7424
Epoch 3885/4000
1545/1545 [==============================] - 0s 187us/step - loss: 0.6067 - acc: 0.7333
Epoch 3886/4000
1545/1545 [==============================] - 0s 165us/step - loss: 0.6076 - acc: 0.7346
Epoch 3887/4000
1545/1545 [==============================] - 0s 143us/step - loss: 0.6076 - acc: 0.7346
Epoch 3888/4000
1545/1545 [==============================] - 0s 142us/step - loss: 0.6066 - acc: 0.7366
Epoch 3889/4000
1545/1545 [==============================] - 0s 165us/step - loss: 0.6071 - acc: 0.7366
Epoch 3890/4000
1545/1545 [==============================] - 0s 149us/step - loss: 0.6091 - acc: 0.7385
Epoch 3891/4000
1545/1545 [==============================] - 0s 146us/step - loss: 0.6075 - acc: 0.7372
Epoch 3892/4000
1545/1545 [==============================] - 0s 181us/step - loss: 0.6067 - acc: 0.7353
Epoch 3893/4000
1545/1545 [==============================] - 0s 

Epoch 3963/4000
1545/1545 [==============================] - 0s 175us/step - loss: 0.6072 - acc: 0.7372
Epoch 3964/4000
1545/1545 [==============================] - 0s 145us/step - loss: 0.6076 - acc: 0.7372
Epoch 3965/4000
1545/1545 [==============================] - 0s 185us/step - loss: 0.6067 - acc: 0.7398
Epoch 3966/4000
1545/1545 [==============================] - 0s 239us/step - loss: 0.6074 - acc: 0.7353
Epoch 3967/4000
1545/1545 [==============================] - 0s 178us/step - loss: 0.6067 - acc: 0.7372
Epoch 3968/4000
1545/1545 [==============================] - 1s 330us/step - loss: 0.6058 - acc: 0.7359
Epoch 3969/4000
1545/1545 [==============================] - 1s 403us/step - loss: 0.6058 - acc: 0.7372
Epoch 3970/4000
1545/1545 [==============================] - 0s 217us/step - loss: 0.6082 - acc: 0.7346
Epoch 3971/4000
1545/1545 [==============================] - 0s 256us/step - loss: 0.6070 - acc: 0.7417
Epoch 3972/4000
1545/1545 [==============================] - 0s 

In [22]:
lossList=[]
for i in range(100):
    loss_value=model.evaluate([train_latent, train_PTcom, train_CARcom, train_PTasc, train_CARasc, train_SMasc], 
          train_y, batch_size=train_latent.shape[0],verbose=0)[0]
    lossList.append(loss_value)
print(np.mean(lossList)*train_latent.shape[0])

937.7965884268284


In [58]:
model.predict([train_latent[:1], train_PTcom[:1], train_CARcom[:1], train_PTasc[:1], train_CARasc[:1], train_SMasc[:1]])

array([[5.6687373e-01, 4.3312630e-01, 8.0049547e-13]], dtype=float32)

In [39]:
from tensorflow.keras.utils import plot_model
plot_model(idmodel, to_file='linearmodel.png',show_shapes=True)

# Debug part

In [23]:
i = 0
for layer in model.layers:
    print(layer.name, i)
    i += 1

input_latent 0
car_sys 1
input_PTcom 2
input_CARcom 3
carlover 4
input_PTasc 5
util_common 6
input_CARasc 7
input_SMasc 8
v0_as 9
trans_to_rand_shape 10
v1_as 11
trans_to_rand_shape_1 12
v2_as 13
V0 14
V1 15
V2 16
prediction 17


In [18]:
model.layers[9].get_weights()

[array([[-1.2084553]], dtype=float32), array([[-0.57047397]], dtype=float32)]

## LatentPart

In [25]:
for i in range(len(model.layers)):
    print('{0:15s} {1}'.format(model.layers[i].name, model.layers[i].get_weights()))

input_latent    []
car_sys         [array([[ 0.10536959],
       [-0.01933521],
       [ 0.01026487],
       [ 0.02635694],
       [ 0.22914945],
       [-0.09846406],
       [ 0.07474083],
       [ 0.62859607],
       [ 0.23671116],
       [ 0.13821891],
       [-0.02835677],
       [ 0.2926643 ],
       [-0.5817164 ]], dtype=float32), array([1.912908], dtype=float32)]
input_PTcom     []
input_CARcom    []
carlover        []
input_PTasc     []
util_common     [array([[-0.14692418],
       [-0.04842361]], dtype=float32)]
input_CARasc    []
input_SMasc     []
v0_as           [array([[-0.01188811]], dtype=float32), array([[2.6095166]], dtype=float32), array([[-0.03894015]], dtype=float32)]
trans_to_rand_shape []
v1_as           [array([[8.188059]], dtype=float32), array([[-0.03715079]], dtype=float32)]
trans_to_rand_shape_1 []
v2_as           [array([[-0.40190613]], dtype=float32), array([[-4.1542125]], dtype=float32), array([[4.1361713]], dtype=float32)]
V0              []
V1           

In [47]:
model.layers[9].get_weights()

[array([[-1.392449]], dtype=float32), array([[-0.55060613]], dtype=float32)]